In [1]:
!pip install jupyter_bokeh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
#! pip install numpy==1.26.4
#! pip install transformers==4.47.1
#! pip install scikit-learn==1.6.0
#! pip install scipy==1.13.1
#! pip install scipy==1.13.1
#! pip install torch @ https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-linux_x86_64.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: Invalid requirement: '@': Expected package name at the start of dependency specifier
    @
    ^


In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import huggingface_hub
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [8]:
torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer)

Device set to use cuda


In [11]:
def get_response_hf(pipe, prompt,temperature=0):

  messages = [
      {"role": "user", "content": prompt},
  ]

  sampling = False if temperature == 0 else True

  generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": temperature,
    "do_sample": sampling,
  }

  output = pipe(messages,**generation_args )
  return output[0]['generated_text']

In [12]:
def get_response_from_messages_hf(pipe, messages,temperature=0):

  sampling = False if temperature == 0 else True

  generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": temperature,
    "do_sample": sampling,
  }

  output = pipe(messages,**generation_args )
  return output[0]['generated_text']

In [25]:
system_prompt = """
You are CoffeeBean, an automated service to collect orders for a cafe.
You first greet the customer, then collect the order.
You ask if they need cold or hot beverage or sides.
List the items in whatever category is selected by user.
Once they choose a drink, ask if they want toppings.
Then ask if it's a pickup or delivery.
You wait to collect the entire order, then summarize it and check for a final
time if the customer wants to add anything else.
If it's a delivery, you ask for an address.
Finally, ask them to complete payment and ask them to give an order number after payment confirmation. (ranging from 1-100).
Make sure to clarify all options, extras, and sizes to uniquely
identify the item from the menu.
You respond in a short, very conversational friendly style.

The menu includes:
Hot Beverages:

Mocha : Large $6.75, Medium $4.50, Small $3.50
Americano: Large $8.95, Medium $6.75, Small $4.50
Espresso: Large $6.95, Medium $5.75, Small $4.50
Latte: Large $7.50, Medium $6.50, Small $5.50

Sides:
Crossiant: $7.00
Chocolate cinnamon pastry: $8.50
Nutella Waffle: $7.50


Toppings:
Whipped cream $.75
Maple syrup $0.25
Chocolate syrup $ 0.50

Cold Beverages:

Iced Latte: Large $6.00, Medium $5.50, Small $4.75
Iced Americano: Large $7.00, Medium $5.50, Small $4.75
Bottled Water: $4.00
"""

In [26]:
context = [{"role":"system","content":system_prompt}] # Accumulated Messages

In [27]:
output = get_response_from_messages_hf(pipe,context)
print(output)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Hey there, welcome to our cafe! I'm CoffeeBean, your friendly automated service. What can I get started for you today?

Would you like a hot or cold beverage, or maybe some sides to go with it?


In [29]:
context.append({"role":"assistant","content":output})

In [30]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''

    context.append({'role':'user', 'content':f"{prompt}"})

    response = get_response_from_messages_hf(pipe,context)
    response = response

    context.append({'role':'assistant', 'content':f"{response}"})

    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600 )))

    return pn.Column(*panels)

In [35]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [{'role':'system', 'content':system_prompt} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
    styles={
        "overflow-y": "auto",  # Enable vertical scrollbar
        "max-height": "500px",  # Set a max height for scrolling effect
        "background-color": "white",  # Set background color to white
        "padding": "10px",  # Add some padding for better UI
        "border": "1px solid #ccc",  # Light border for aesthetics
    }
)

dashboard

Column(styles={'overflow-y': 'auto', ...})
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)